In [1]:
#### Librerias


import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pyvis import network as net
from pyvis.network import Network
from networkx.algorithms import community
import networkx.algorithms.community as nxcom
import community as community_louvain

%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

In [2]:
### Datos de entrada

def network(Base):
    data = pd.read_excel(Base)
    data = data[['X1','X4','X6','X9','X13','X24']]
    data['X1'] = data['X1'].replace(0, 'Cero')
    data['X1'] = data['X1'].str.strip()
    data['X4'] = data['X4'].str.strip()
    data['X6'] = data['X6'].str.strip()
    data['X9'] = data['X9'].str.strip()
    data['X13'] = data['X13'].str.strip()
    data['X24'] = data['X24'].str.strip()
    
    
    def normalize(s):
        replacements = (("á", "a"),("é", "e"),("í", "i"),("ó", "o"),("ú", "u"),('ñ', 'n'))
        for a, b in replacements:
            s = s.replace(a, b).replace(a.upper(), b.upper())
        return s
    
    data['X1'] = data['X1'].apply(normalize)
    data['X4'] = data['X4'].apply(normalize)
    data['X6'] = data['X6'].apply(normalize)
    data['X9'] = data['X9'].apply(normalize)
    data['X13'] = data['X13'].apply(normalize)
    data['X24'] = data['X24'].apply(normalize)

    a = data[['X1','X6']].rename({'X1': 'Nodo','X6':'Edge'}, axis = 1)
    b = data[['X4','X6']].rename({'X4': 'Nodo','X6':'Edge'}, axis = 1)
    c = data[['X9','X4']].rename({'X9': 'Nodo','X4':'Edge'}, axis = 1)
    d = data[['X9','X13']].rename({'X9': 'Nodo','X13':'Edge'}, axis = 1)
    #e = data[['X24','X13']].rename({'X24': 'Nodo','X13':'Edge'}, axis = 1)
    
    Frames = [a, b, c, d]
    result = pd.concat(Frames)
    
    red = nx.from_pandas_edgelist(result,
                                  source = "Nodo",
                                  target = "Edge")
    
    print('#Nodo entrada (nodes):', len(red.nodes()), 'y', '#Nodo salida (edges):', len(red.edges()))
    
    return(result, red)

In [3]:
result, red = network(r'Mayo_2023.xlsx')
result.shape

#Nodo entrada (nodes): 687 y #Nodo salida (edges): 1268


(2464, 2)

In [5]:
def Stadis_Netw(red):
    in_degree_centrality = pd.DataFrame(nx.in_degree_centrality(red).items(),
                                        columns=["Nodo",
                                                 "In_DgC"]) # In degree centrality
    out_degree_centrality = pd.DataFrame(nx.out_degree_centrality(red).items(), 
                                       columns=["Nodo", "Ot_DgC"]) # In degree centrality
    closeness_centrality = pd.DataFrame(nx.closeness_centrality(red).items(), 
                                       columns=["Nodo", "Clos_C"])  # Closeness centrality
    betweenness_centrality = pd.DataFrame(nx.betweenness_centrality(red).items(), 
                                       columns=["Nodo", "Btw_C"])  # Betweenness centrality
    pagerank_centrality = pd.DataFrame(nx.pagerank(red).items(),
                                       columns=["Nodo", "Pagerank_C"])
    hub_scores, auth_scores = nx.hits(red)
    hub_centrality = pd.DataFrame(hub_scores.items(),
                                  columns=["Nodo", "Hub_C"]) # Hub centrality
    auth_centrality = pd.DataFrame(auth_scores.items(),
                                   columns=["Nodo", "Auth_C"]) # Auth centrality
    centrality_summary = in_degree_centrality\
                         .merge(out_degree_centrality, on = "Nodo")\
                         .merge(closeness_centrality, on = "Nodo")\
                         .merge(betweenness_centrality, on = "Nodo")\
                         .merge(pagerank_centrality, on = "Nodo")\
                         .merge(hub_centrality, on = "Nodo")\
                         .merge(auth_centrality, on = "Nodo")
    return (centrality_summary)

In [23]:
#### Modificación de sintaxis 

color_list = ["#e1a793", "#506fac", "#d2bf84", "#73a0dd", "#ad874f", "#78c8e4",
              "#b0637c", "#7bdbcd", "#a46d5a", "#53a785", "#cfa6e2", "#5d854d",
              "#e19db5", "#4f7c70", "#8c6f9b", "#a9c99c", "#4e82a5", "#7f7944", "#b5b8dd", "#5ba6ae"]


### Comunidad extraida de grafico sin direcciones 
red_no_direct = nx.from_pandas_edgelist(result, source = "Nodo", target = "Edge")

### Se calcula la comunidad en el grafico no dirigido
partition_object = community_louvain.best_partition(red_no_direct, random_state = 123)

### Se crea un grafico dirigido 

Red_digra = pd.DataFrame(result.groupby(['Nodo'])['Edge'].nunique()).rename({'Edge': 'Weight'}, axis = 1).reset_index()
Di_grap = result.merge(Red_digra, on = 'Nodo')
Di_grap['Weight'] = Di_grap['Weight'].astype(float)
Di_grap['Weight'] = (Di_grap['Weight'] /Di_grap['Weight'].abs().max())*5

red_Si_direct = nx.from_pandas_edgelist(Di_grap, source = "Nodo", target = "Edge", create_using = nx.DiGraph())

### Los valores calculados de las comunidades se agregan al grafico con direcciones
values = [partition_object.get(node) for node in red_Si_direct.nodes()]

color_list = color_list[0:len(set(values))]

color_dict = pd.Series(color_list, index=np.arange(0,len(set(values)))).to_dict()

for key, value in partition_object.items():partition_object[key] = color_dict[value]
    
nx.set_node_attributes(red_Si_direct, partition_object, 'color')


## Se calculan los pesos de los nodos en la red direccionada 
sizes = [n for n in list(dict(nx.degree(red_Si_direct)).values())]

#### Se crea el grafico 
vis_g = net.Network(notebook = False,
                    directed = True,
                    #filter_menu=True,
                    cdn_resources='remote',
                    select_menu = True,
                    bgcolor = "#1b1b1b",
                    font_color = "white")


vis_g.from_nx(red_Si_direct)
vis_g.barnes_hut(gravity = -8600, 
                 central_gravity = 0.1,
                 spring_length = 200,
                 spring_strength = 0.03,
                 damping = 0.13,
                 overlap = 0.13)

vis_g.set_edge_smooth('diagonalCross')


#vis_g.show_buttons(filter_=['physics']) 


for n in range(len(vis_g.nodes)):
    vis_g.nodes[n]["size"]=sizes[n]
    
#for n in range(len(vis_g.edges)):
 #   vis_g.edges[n]["width"]=Di_grap["Weight"][n]
    
vis_g.show("Prueba_03.html") 

In [10]:
def Network_CECTI(data):
    
    color_list = ["#e1a793", "#506fac", "#d2bf84", "#73a0dd", "#ad874f", "#78c8e4",
                  "#b0637c", "#7bdbcd", "#a46d5a", "#53a785", "#cfa6e2", "#5d854d",
                  "#e19db5", "#4f7c70", "#8c6f9b", "#a9c99c", "#4e82a5", "#7f7944", "#b5b8dd", "#5ba6ae"]
    
    ### Comunidad extraida de grafico sin direcciones 
    red_no_direct = nx.from_pandas_edgelist(data, source = "Nodo", target = "Edge")
    
    ### Se calcula la comunidad en el grafico no dirigido
    partition_object = community_louvain.best_partition(red_no_direct, random_state = 123)

    
    ### Se crea un grafico dirigido 
    
    Red_digra = pd.DataFrame(data.groupby(['Nodo'])['Edge'].nunique()).rename({'Edge': 'Weight'}, axis = 1).reset_index()
    Di_grap = result.merge(Red_digra, on = 'Nodo')
    Di_grap['Weight'] = Di_grap['Weight'].astype(float)
    Di_grap['Weight'] = (Di_grap['Weight'] /Di_grap['Weight'].abs().max())*5
    
    red_Si_direct = nx.from_pandas_edgelist(Di
                                            _grap, source = "Nodo", target = "Edge", create_using = nx.DiGraph())
    
    ### Los valores calculados de las comunidades se agregan al grafico con direcciones
    values = [partition_object.get(node) for node in red_Si_direct.nodes()]
    
    print('# Total de Comunidads:', len(set(values)))
    
    color_list = color_list[0:len(set(values))]

    color_dict = pd.Series(color_list, index=np.arange(0,len(set(values)))).to_dict()
    
    for key, value in partition_object.items():partition_object[key] = color_dict[value]
        
    nx.set_node_attributes(red_Si_direct, partition_object, 'color')
    
    ## Se calculan los pesos de los nodos en la red direccionada 
    sizes = [n for n in list(dict(nx.degree(red_Si_direct)).values())]
    
    #### Se crea el grafico 
    vis_g = net.Network(notebook = False,
                        directed = True,
                        cdn_resources='remote',
                        select_menu = True,
                        bgcolor = "#1b1b1b",
                        font_color = "white")
    
    vis_g.from_nx(red_Si_direct)
    vis_g.barnes_hut(gravity = -8600,
                     central_gravity = 0.1,
                     spring_length = 200,
                     spring_strength = 0.03,
                     damping = 0.13,
                     overlap = 0.13)
    
    vis_g.set_edge_smooth('diagonalCross')
    
    for n in range(len(vis_g.nodes)):
        vis_g.nodes[n]["size"]=sizes[n]
        
    #for n in range(len(vis_g.edges)):
    #vis_g.edges[n]["width"]=Di_grap["Weight"][n]
    
    vis_g.show("Modelo_red.html")
    
    return (red_Si_direct)

In [11]:
red_Si_direct = Network_CECTI(result)

# Total de Comunidads: 15


In [12]:
centrality_summary = Stadis_Netw(red_Si_direct)
centrality_summary.sort_values(by=['Pagerank_C'], ascending=False)

,Nodo,In_DgC,Ot_DgC,Clos_C,Btw_C,Pagerank_C,Hub_C,Auth_C
2,Por definir,0.023324,0.001458,0.214797,0.000000,0.146508,2.263995e-08,1.315170e-06
7,AGCTI,0.046647,0.000000,0.322220,0.000000,0.028688,-0.000000e+00,1.631865e-06
48,Ignacio Ramon Ruiz Casillas,0.135569,0.010204,0.135569,0.001377,0.025186,1.059617e-07,-2.027647e-15
1,AGSC,0.037901,0.000000,0.342843,0.000000,0.023874,-0.000000e+00,1.581330e-06
6,AGR,0.037901,0.000000,0.349012,0.000000,0.023595,-0.000000e+00,6.429813e-07
...,...,...,...,...,...,...,...,...
280,PORTAL_PUBLICO,0.000000,0.002915,0.000000,0.000000,0.000628,1.597664e-06,-0.000000e+00
281,RECEPCION CFDI,0.000000,0.002915,0.000000,0.000000,0.000628,1.597664e-06,-0.000000e+00
282,VAD,0.000000,0.002915,0.000000,0.000000,0.000628,-3.621207e-17,-0.000000e+00
283,DIAMANTES,0.000000,0.002915,0.000000,0.000000,0.000628,5.359219e-18,-0.000000e+00


In [15]:
centrality_ranks = centrality_summary.rank(ascending = False, 
                                           numeric_only = True, 
                                           method = "dense").astype(int)
centrality_ranks.insert(loc=0, column='Nodo', value=centrality_summary["Nodo"])
centrality_rank = centrality_ranks.sort_values(by=['Clos_C']).head(10).style.background_gradient(cmap='inferno',
                                                                                                  subset=list(centrality_ranks.columns[1:]))
centrality_rank

,Nodo,In_DgC,Ot_DgC,Clos_C,Btw_C,Pagerank_C,Hub_C,Auth_C
6,AGR,12,10,1,21,5,99,36
1,AGSC,12,10,2,21,4,99,29
7,AGCTI,9,10,3,21,2,99,28
2,Por definir,17,9,4,21,1,64,30
8,AGP,18,10,5,21,9,99,38
4,AGAFF,21,10,6,21,12,99,37
22,AGJ,24,10,7,21,15,99,40
5,AGGC,23,10,8,21,16,99,39
48,Ignacio Ramon Ruiz Casillas,1,3,9,1,3,33,106
52,Alejandro Granados Lujano,2,3,10,2,6,41,1


In [16]:
average_ranks = pd.DataFrame(round(centrality_ranks.mean(axis=1)).astype(int), columns=["average_rank"])
average_ranks.insert(loc = 0, column='Nodo', value=centrality_summary["Nodo"])
average_rank = average_ranks.sort_values(by = ["average_rank"]).reset_index(drop=True)\
    .head(10).style.background_gradient(cmap='inferno', subset=["average_rank"])
average_rank

,Nodo,average_rank
0,Alejandro Granados Lujano,9
1,Arnulfo Melendez Valdes,12
2,Enrique Teodoro Olvera Olvera,12
3,Jesus Guerrero Mendoza,17
4,Alfredo Reyes Luna,18
5,Alejandro Federico Tejeda Munoz,18
6,Por definir,21
7,Ignacio Ramon Ruiz Casillas,22
8,Jorge Alberto De Flon Gonzalez,23
9,AGSC,25
